In [42]:
import psycopg2
from psycopg2 import sql
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone
from dateutil import tz

import os
import sys 

from configupdater import ConfigUpdater
# pip install ConfigUpdater

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

# [metadata]
# pmr_pm_plan = 2019-01-01 00:00:00
# pmr_pm_item = 2019-01-01 00:00:00
# pmr_project = 2019-01-01 00:00:00
# pmr_inventory  = 2019-01-01 00:00:00


# Imported Library

In [43]:
is_py=False
view_name = "pmr_inventory"
isFirstLoad=False
if is_py:
    press_Y=''
    ok=False

    if len(sys.argv) > 1:
        view_name=sys.argv[1]
    else:
        print("Enter the following input: ")
        view_name = input("View Table Name : ")
print(f"View name to load to BQ :{view_name}")

View name to load to BQ :pmr_inventory


# Imported date

In [44]:
dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
print(f"UTC: {dt_imported} For This Import")



UTC: 2023-12-31 08:12:24 For This Import


# Set view data and log table

In [45]:
log = "models_logging_change"
def get_contentID_keyName(view_name):

    if view_name == "pmr_pm_plan":
        tableContentID = 36
        key_name = "pm_id"
        sp="merge_pm_plan"
        changed_field_mapping=['planned_date','ended_pm_date',
                               'project_id','remark','team_lead_id']

    elif view_name == "pmr_pm_item":
        tableContentID = 37
        key_name = "pm_item_id"
        sp="merge_pm_item"

    elif view_name == "pmr_project":
        tableContentID = 7
        key_name = "project_id"
        sp="merge_project"

    elif view_name == "pmr_inventory":
        tableContentID = 14
        key_name = "inventory_id"
        sp="merge_inventory"
        hanged_field_mapping=[
        ]

    else:
        raise Exception("No specified content type id")
        
    return tableContentID, key_name,sp


content_id , view_name_id,sp_name=get_contentID_keyName(view_name)
print(content_id," - ",view_name_id," - ",sp_name)

14  -  inventory_id  -  merge_inventory


# Set data and cofig path

In [46]:
# Test config,env file and key to be used ,all of used key  are existing.
cfg_path="cfg_last_import"
env_path='.env'

updater = ConfigUpdater()
updater.read(os.path.join(cfg_path,f"{view_name}.cfg"))

config = dotenv_values(dotenv_path=env_path)

print(env_path)
print(cfg_path)

.env
cfg_last_import


In [47]:
# Test exsitng project dataset and table anme

projectId=config['PROJECT_ID']  # smart-data-ml  or kku-intern-dataai or ponthorn
credential_file=config['PROJECT_CREDENTIAL_FILE']
# C:\Windows\smart-data-ml-91b6f6204773.json
# C:\Windows\kku-intern-dataai-a5449aee8483.json
# C:\Windows\pongthorn-5decdc5124f5.json


dataset_id='SMartData_Temp'  # 'SMartData_Temp'  'PMReport_Temp'
main_dataset_id='SMartDataAnalytics'  # ='SMartDataAnalytics'  'PMReport_Main'

credentials = service_account.Credentials.from_service_account_file(credential_file)

table_name=view_name.replace("pmr_","temp_") #can change in ("name") to temp table
table_id = f"{projectId}.{dataset_id}.{table_name}"
print(table_id)


main_table_name=view_name.replace("pmr_","")
main_table_id = f"{projectId}.{main_dataset_id}.{main_table_name}"
print(main_table_id)

# https://cloud.google.com/bigquery/docs/reference/rest/v2/Job
to_bq_mode="WRITE_EMPTY"


client = bigquery.Client(credentials= credentials,project=projectId)

pongthorn.SMartData_Temp.temp_inventory
pongthorn.SMartDataAnalytics.inventory


Read Configuration File and Initialize BQ Object

In [48]:
last_imported=datetime.strptime(updater["metadata"][view_name].value,"%Y-%m-%d %H:%M:%S")
print(f"UTC:{last_imported}  Of Last Import")

# local_zone = tz.tzlocal()
# last_imported = last_imported.astimezone(local_zone)
# print(f"Local Asia/Bangkok:{last_imported}")

UTC:2019-01-01 00:00:00  Of Last Import


# Postgres &BigQuery

In [49]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [50]:
def get_bq_table():
 try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
    return True
 except NotFound:
    raise Exception("Table {} is not found.".format(table_id))
    

def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(write_disposition=to_bq_mode,)
        job = client.load_table_from_dataframe(df_trasns, table_id, job_config=job_config)
        try:
         job.result()  # Wait for the job to complete.
        except ClientError as e:
         print(job.errors)

        print("Total ", len(df_trasns), f"Imported data to {table_id} on bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

# Check whether it is the first loading?

In [51]:
def checkFirstLoad():
    print("If the main table is empty , so the action of each row  must be 'added' on temp table")
    rows_iter   = client.list_rows(main_table_id, max_results=1) 
    no_main=len(list(rows_iter))
    if no_main==0:
     isFirstLoad=True
     print(f"This is the first loaing , so there is No DATA in {main_table_id}, we load all rows from {view_name} to import into {table_id} action will be 'added' ")
    else:
     isFirstLoad=False   
    return isFirstLoad

In [52]:
isFirstLoad=checkFirstLoad()
print(f"IsFirstLoad={isFirstLoad}")

If the main table is empty , so the action of each row  must be 'added' on temp table
This is the first loaing , so there is No DATA in pongthorn.SMartDataAnalytics.inventory, we load all rows from pmr_inventory to import into pongthorn.SMartData_Temp.temp_inventory action will be 'added' 
IsFirstLoad=True


# For The next Load
* get data from model log based on condition last_imported and table
* Get all actions from log table by selecting unique object_id and setting by doing something as logic
* Create  id and action dataframe form filtered rows from log table

In [53]:
def list_model_log(x_last_imported,x_content_id):
    sql_log = f"""
    SELECT object_id, action,TO_CHAR(date_created,'YYYY-MM-DD HH24:MI:SS') as date_created ,changed_data
    FROM {log}
    WHERE date_created  AT time zone 'utc' >= '{x_last_imported}' AND content_type_id = {x_content_id} 
    ORDER BY object_id, date_created
    """
    print(sql_log)


    # Asia/Bangkok 
    lf = list_data(sql_log, None, get_postgres_conn())
    print(f"Retrieve all rows after {last_imported}")
    print(lf.info())
    return lf


In [54]:
def check_any_changes_to_collumns_view(dfAction,x_view_name,_x_key_name):
    """
    Check dataframe from log model that contain only changed action to select changed fields on view.
    """

    listACtion=dfAction["action"].unique().tolist()
    if len(listACtion)==1 and listACtion[0]=='changed':
        print("###########################################################")
        print("Process dataframe containing only all changed action")
        print(dfAction)
        print("###########################################################")
    
    

In [55]:
def select_actual_action(lf):
    listIDs=lf["object_id"].unique().tolist()
    listUpdateData=[]
    for id in listIDs:
        lfTemp=lf.query("object_id==@id")
        print(lfTemp)
        print("----------------------------------------------------------------")
        
        
        # check_any_changes_to_collumns_view(lfTemp,content_id,view_name_id)


        first_row = lfTemp.iloc[0]
        last_row = lfTemp.iloc[-1]
        # print(first_row)
        # print(last_row)

        if len(lfTemp)==1:
            listUpdateData.append([id,first_row["action"]])
        else:
            if first_row["action"] == "added" and last_row["action"] == "deleted":
                continue
            elif first_row["action"] == "added" and last_row["action"] != "deleted":
                listUpdateData.append([id,"added"])
            else : listUpdateData.append([id,last_row["action"]])

    print("Convert listUpdate to dataframe")
    dfUpdateData = pd.DataFrame(listUpdateData, columns= ['id', 'action'])
    dfUpdateData['id'] = dfUpdateData['id'].astype('int64')
    dfUpdateData=dfUpdateData.sort_values(by="id")
    dfUpdateData=dfUpdateData.reset_index(drop=True)

    return dfUpdateData


In [56]:
if isFirstLoad==False:
    listModelLogObjectIDs=[]
    dfModelLog=list_model_log(last_imported,content_id)
    if dfModelLog.empty==True:
        print("No row to be imported.")
        exit()
    else:
       print("Get row imported from model log to set action") 
       dfModelLog=select_actual_action( dfModelLog)
       listModelLogObjectIDs=dfModelLog['id'].tolist()
       print(dfModelLog.info())
       print(dfModelLog)       
       print(listModelLogObjectIDs) 

# Load view and transform

In [57]:
def retrive_next_data_from_view(x_view,x_id,x_listModelLogObjectIDs):
    if len(x_listModelLogObjectIDs)>1:
     sql_view=f"select *  from {x_view}  where {x_id} in {tuple(x_listModelLogObjectIDs)}"
    else:
     sql_view=f"select *  from {x_view}  where {x_id} ={x_listModelLogObjectIDs[0]}"
    
    print(sql_view)
    df=list_data(sql_view,None,get_postgres_conn())

    if df.empty==True:
     return df
    df=df.drop(columns='updated_at')
    return df 


def retrive_first_data_from_view(x_view,x_last_imported):
     sql_view=f"select *  from {x_view}  where  updated_at AT time zone 'utc' >= '{x_last_imported}'"
     print(sql_view)
     df=list_data(sql_view,None,get_postgres_conn())
     if df.empty==True:
            return df
     df=df.drop(columns='updated_at')
     df['action']='added'
     return df   
def retrive_one_row_from_view_to_gen_df_schema(x_view):
    sql_view=f"select *  from {x_view}  limit 1"
    print(sql_view)
    df=list_data(sql_view,None,get_postgres_conn())
    df=df.drop(columns='updated_at')
    return df


if isFirstLoad:
 df=retrive_first_data_from_view(view_name,last_imported)
 if df.empty==True:
    print("No row to be imported.")
    exit()
 else:
    print(df.info())
else:
 df=retrive_next_data_from_view(view_name,view_name_id,listModelLogObjectIDs)  
 if df.empty==True:
    print("Due to having deleted items, we will Get schema from {} to create empty dataframe with schema.")
    df=retrive_one_row_from_view_to_gen_df_schema(view_name)
    # this id has been included in listModelLogObjectIDs which contain deleted action , so we can use it as schema generation
    print(df)

    
    
    

select *  from pmr_inventory  where  updated_at AT time zone 'utc' >= '2019-01-01 00:00:00'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16399 entries, 0 to 16398
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   inventory_id             16399 non-null  int64 
 1   serial_number            16399 non-null  object
 2   customer_warranty_start  16399 non-null  object
 3   customer_warranty_end    16399 non-null  object
 4   brand                    16399 non-null  object
 5   model                    16399 non-null  object
 6   product_type             16399 non-null  object
 7   project_id               16399 non-null  int64 
 8   action                   16399 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.1+ MB
None


# Data Transaformation
* IF The first load then add actio='Added'
* IF The nextload then Merge LogDF and ViewDF and add deleted row 
  * Get Deleted Items  to Create deleted dataframe by using listDeleted
  * If there is one deletd row then  we will merge it to master dataframe
* IF the next load has only deleted action

In [58]:
def add_acutal_action_to_df_at_next(df,dfUpdateData,x_view,x_id):
    # merget model log(id and action) to data view
    # if  dfUpdateData  contain only deleted action
    # we will merge to get datafdame shcema, it can perform inner without have actual data fram view
    merged_df = pd.merge(df, dfUpdateData, left_on=view_name_id, right_on='id', how='inner')
    merged_df = merged_df.drop(columns=['id'])

    listAllAction=dfUpdateData['id'].tolist()
    print(f"List {listAllAction} all action")
    
    listSeleted = merged_df[view_name_id].tolist()
    print(f"List  {x_view}   {listSeleted} from {x_view} exluding deleted action")
    
    allActionSet = set(listAllAction)
    anotherSet = set(listSeleted)
    
    listDeleted = list(allActionSet.symmetric_difference(anotherSet))
    print(f"List deleted {listDeleted}")
    
    # Test List  select by view + List deeleted = List All Action

    if len(listDeleted)>0:
        print("There are some deleted rows")
        dfDeleted=pd.DataFrame(data=listDeleted,columns=[view_name_id])
        dfDeleted['action']='deleted'
        print(dfDeleted)
        merged_df=pd.concat([merged_df,dfDeleted],axis=0)

    else:
        print("No row deleted")

    return merged_df    




In [59]:
if isFirstLoad==False:
 df=add_acutal_action_to_df_at_next(df,dfModelLog,view_name,view_name_id)

print(df)

       inventory_id  serial_number customer_warranty_start  \
0                47   451951000035              2020-01-12   
1                46   451915000214              2019-04-26   
2                18   451741000112              2017-10-25   
3                45   451915000213              2019-04-26   
4                44   451902000028              2019-01-23   
...             ...            ...                     ...   
16394         19205    FDO23011B4G              2020-07-04   
16395         19206    FDO23011AVZ              2020-07-04   
16396         19211           np01              2023-12-01   
16397         19213  win-server-01              2023-12-01   
16398         19214              -              2023-12-01   

      customer_warranty_end      brand                              model  \
0                2025-01-12     NetApp                           AFF A300   
1                2024-04-29     NetApp                           AFF A300   
2                2022-10

# Last Step :Check duplicate ID & reset index

In [60]:
hasDplicateIDs = df[view_name_id].duplicated().any()
if  hasDplicateIDs:
 raise Exception("There are some duplicate id on dfUpdateData")
else:
 print(f"There is no duplicate {view_name_id} ID")  


# merged_df['imported_at']=dt_imported
df=df.reset_index(drop=True  )
print(df.info())
print(df)

There is no duplicate inventory_id ID
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16399 entries, 0 to 16398
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   inventory_id             16399 non-null  int64 
 1   serial_number            16399 non-null  object
 2   customer_warranty_start  16399 non-null  object
 3   customer_warranty_end    16399 non-null  object
 4   brand                    16399 non-null  object
 5   model                    16399 non-null  object
 6   product_type             16399 non-null  object
 7   project_id               16399 non-null  int64 
 8   action                   16399 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.1+ MB
None
       inventory_id  serial_number customer_warranty_start  \
0                47   451951000035              2020-01-12   
1                46   451915000214              2019-04-26   
2                18   45174100

In [61]:
df

,inventory_id,serial_number,customer_warranty_start,customer_warranty_end,brand,model,product_type,project_id,action
0,47,451951000035,2020-01-12,2025-01-12,NetApp,AFF A300,Storage,22,added
1,46,451915000214,2019-04-26,2024-04-29,NetApp,AFF A300,Storage,20,added
2,18,451741000112,2017-10-25,2022-10-31,NetApp,AFF A300,Storage,11,added
3,45,451915000213,2019-04-26,2024-04-29,NetApp,AFF A300,Storage,20,added
4,44,451902000028,2019-01-23,2024-01-22,NetApp,FAS8200,Storage,19,added
...,...,...,...,...,...,...,...,...,...
16394,19205,FDO23011B4G,2020-07-04,2024-07-03,Cisco,Nexus 9300,Switch,721,added
16395,19206,FDO23011AVZ,2020-07-04,2024-07-03,Cisco,Nexus 9300,Switch,721,added
16396,19211,np01,2023-12-01,2024-12-31,NetApp,16Pt 10Gb-C,Storage,765,added
16397,19213,win-server-01,2023-12-01,2024-12-31,Microsoft,Microsoft Windows Server Standard,Server,765,added


# Insert data to BQ data frame

In [62]:
if get_bq_table():
    try:
        insertDataFrameToBQ(df)
    except Exception as ex:
        raise ex

Table pongthorn.SMartData_Temp.temp_inventory already exists.
[SchemaField('inventory_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('serial_number', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('customer_warranty_start', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('customer_warranty_end', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('brand', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('model', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('product_type', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('project_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('action', 'STRING', 'NULLABLE', None, None, (), None)]
Total  16399 Imported data to pongthorn.SMartData_Temp.temp_inventory on bigquery successfully


# Run StoreProcedure To Merge Temp&Main and Truncate Transaction 

In [63]:
print("# Run StoreProcedure To Merge Temp&Main and Truncate Transaction.")
# https://cloud.google.com/bigquery/docs/transactions
sp_id_to_invoke=f""" CALL `{projectId}.{main_dataset_id}.{sp_name}`() """
print(sp_id_to_invoke)

sp_job = client.query(sp_id_to_invoke)


# Run StoreProcedure To Merge Temp&Main and Truncate Transaction.
 CALL `pongthorn.SMartDataAnalytics.merge_inventory`() 


In [64]:
updater["metadata"][view_name].value=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
updater.update_file() 

<ConfigUpdater [
    <Section: 'metadata' [
        <Option: pmr_inventory = '2023-12-31 08:12:24'>
    ]>
]>

In [65]:
print(datetime.now(timezone.utc) )

2023-12-31 08:12:32.364800+00:00
